In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
from datetime import datetime

# Base class for bank accounts
class BankAccount:
    """Base class for bank accounts."""

    def __init__(self, account_number, account_holder, initial_balance=0):
        """
        Initialize a new bank account.
        :param account_number: Unique identifier for the account.
        :param account_holder: Name of the account holder.
        :param initial_balance: Starting balance of the account (default is 0).
        """
        self._account_number = account_number  # Protected attribute for account number
        self._account_holder = account_holder  # Protected attribute for account holder
        self._balance = initial_balance  # Protected attribute for balance
        self._transaction_history = []  # List to store transaction history

    def deposit(self, amount):
        """
        Deposit money into the account.
        :param amount: Amount to deposit (must be positive).
        :return: A message indicating success or failure.
        """
        if amount > 0:
            self._balance += amount  # Add amount to balance
            self._transaction_history.append(
                f"Deposit: +${amount:.2f} | Balance: ${self._balance:.2f} | {datetime.now()}"
            )  # Record transaction
            return f"${amount:.2f} deposited successfully. New balance: ${self._balance:.2f}"
        else:
            return "Invalid deposit amount."

    def withdraw(self, amount):
        """
        Withdraw money from the account if sufficient balance.
        :param amount: Amount to withdraw (must be positive and <= balance).
        :return: A message indicating success or failure.
        """
        if amount > 0 and self._balance >= amount:
            self._balance -= amount  # Subtract amount from balance
            self._transaction_history.append(
                f"Withdrawal: -${amount:.2f} | Balance: ${self._balance:.2f} | {datetime.now()}"
            )  # Record transaction
            return f"${amount:.2f} withdrawn successfully. New balance: ${self._balance:.2f}"
        else:
            return "Insufficient funds or invalid amount."

    def view_balance(self):
        """
        Display the current balance.
        :return: A string with the current balance.
        """
        return f"Current balance: ${self._balance:.2f}"

    def view_transactions(self):
        """
        Show transaction history.
        :return: A string with all transactions.
        """
        return "\n".join(self._transaction_history)

    def transfer(self, other_account, amount):
        """
        Transfer money to another account.
        :param other_account: The account to transfer money to.
        :param amount: Amount to transfer (must be positive and <= balance).
        :return: A message indicating success or failure.
        """
        withdrawal_result = self.withdraw(amount)  # Attempt to withdraw
        if "successfully" in withdrawal_result:
            other_account.deposit(amount)  # Deposit into the other account
            self._transaction_history.append(
                f"Transfer to {other_account._account_number}: -${amount:.2f} | Balance: ${self._balance:.2f} | {datetime.now()}"
            )  # Record transaction
            return f"Transfer of ${amount:.2f} to account {other_account._account_number} successful."
        else:
            return withdrawal_result

    def generate_statement(self):
        """
        Generate a generic account statement.
        :return: A string with account details and transaction history.
        """
        return (
            f"Account Statement for {self._account_holder}\n"
            f"Account Number: {self._account_number}\n"
            f"Account Type: Generic\n"
            f"Current Balance: ${self._balance:.2f}\n\n"
            f"Transaction History:\n{self.view_transactions()}"
        )


# Subclass for savings accounts
class SavingsAccount(BankAccount):
    """Subclass for savings accounts."""

    def generate_statement(self):
        """
        Generate a savings account statement with interest calculation.
        :return: A string with account details, interest earned, and transaction history.
        """
        interest_rate = 0.02  # 2% annual interest
        interest = self._balance * interest_rate  # Calculate interest
        return (
            f"Account Statement for {self._account_holder}\n"
            f"Account Number: {self._account_number}\n"
            f"Account Type: Savings\n"
            f"Current Balance: ${self._balance:.2f}\n"
            f"Interest Earned: ${interest:.2f}\n\n"
            f"Transaction History:\n{self.view_transactions()}"
        )


# Subclass for checking accounts
class CheckingAccount(BankAccount):
    """Subclass for checking accounts."""

    def generate_statement(self):
        """
        Generate a checking account statement with overdraft warning.
        :return: A string with account details, overdraft warning (if applicable), and transaction history.
        """
        overdraft_warning = (
            "Overdraft Warning: Your balance is below $100." if self._balance < 100 else ""
        )  # Add warning if balance is low
        return (
            f"Account Statement for {self._account_holder}\n"
            f"Account Number: {self._account_number}\n"
            f"Account Type: Checking\n"
            f"Current Balance: ${self._balance:.2f}\n"
            f"{overdraft_warning}\n\n"
            f"Transaction History:\n{self.view_transactions()}"
        )


# GUI class for bank account management
class BankAccountGUI:
    def __init__(self, root):
        """
        Initialize the GUI.
        :param root: The root window for the application.
        """
        self.root = root
        self.root.title("Bank Account Management")

        # Create account instances
        self.savings_account = SavingsAccount("123456789", "John Doe", 1000)  # Savings account
        self.checking_account = CheckingAccount("987654321", "Jane Doe", 500)  # Checking account

        # Default account
        self.current_account = self.savings_account

        # GUI Components
        self.label = tk.Label(root, text="Bank Account Management", font=("Arial", 16))
        self.label.pack(pady=10)

        self.balance_label = tk.Label(root, text=self.current_account.view_balance(), font=("Arial", 12))
        self.balance_label.pack(pady=10)

        # Dropdown menu to switch between accounts
        self.account_type_var = tk.StringVar(value="Savings")
        self.account_type_menu = tk.OptionMenu(
            root, self.account_type_var, "Savings", "Checking", command=self.switch_account
        )
        self.account_type_menu.pack(pady=5)

        # Buttons for actions
        self.deposit_button = tk.Button(root, text="Deposit", command=self.deposit)
        self.deposit_button.pack(pady=5)

        self.withdraw_button = tk.Button(root, text="Withdraw", command=self.withdraw)
        self.withdraw_button.pack(pady=5)

        self.transfer_button = tk.Button(root, text="Transfer", command=self.transfer)
        self.transfer_button.pack(pady=5)

        self.statement_button = tk.Button(root, text="Generate Statement", command=self.generate_statement)
        self.statement_button.pack(pady=5)

        self.exit_button = tk.Button(root, text="Exit", command=root.quit)
        self.exit_button.pack(pady=10)

    def switch_account(self, account_type):
        """
        Switch between savings and checking accounts.
        :param account_type: The selected account type ("Savings" or "Checking").
        """
        if account_type == "Savings":
            self.current_account = self.savings_account  # Set current account to savings
        else:
            self.current_account = self.checking_account  # Set current account to checking
        self.update_balance()  # Update the balance label

    def deposit(self):
        """Handle deposit action."""
        amount = self.get_amount("Deposit")  # Get deposit amount from user
        if amount:
            result = self.current_account.deposit(amount)  # Deposit into current account
            messagebox.showinfo("Deposit", result)  # Show result
            self.update_balance()  # Update the balance label

    def withdraw(self):
        """Handle withdrawal action."""
        amount = self.get_amount("Withdraw")  # Get withdrawal amount from user
        if amount:
            result = self.current_account.withdraw(amount)  # Withdraw from current account
            messagebox.showinfo("Withdraw", result)  # Show result
            self.update_balance()  # Update the balance label

    def transfer(self):
        """Handle transfer action."""
        amount = self.get_amount("Transfer")  # Get transfer amount from user
        if amount:
            # For simplicity, transfer between savings and checking accounts
            other_account = (
                self.savings_account if self.current_account == self.checking_account else self.checking_account
            )
            result = self.current_account.transfer(other_account, amount)  # Perform transfer
            messagebox.showinfo("Transfer", result)  # Show result
            self.update_balance()  # Update the balance label

    def generate_statement(self):
        """Generate and display the account statement."""
        statement = self.current_account.generate_statement()  # Generate statement
        messagebox.showinfo("Account Statement", statement)  # Show statement

    def update_balance(self):
        """Update the balance label with the current account's balance."""
        self.balance_label.config(text=self.current_account.view_balance())

    def get_amount(self, action):
        """
        Prompt the user to enter an amount.
        :param action: The action being performed (e.g., "Deposit", "Withdraw").
        :return: The amount entered by the user (or None if invalid).
        """
        amount = simpledialog.askfloat(action, f"Enter amount to {action}:")  # Get amount from user
        if amount is not None and amount > 0:  # Validate amount
            return amount
        else:
            messagebox.showerror("Error", "Invalid amount entered.")  # Show error if invalid
            return None


# Main application
if __name__ == "__main__":
    root = tk.Tk()  # Create the main window
    app = BankAccountGUI(root)  # Initialize the GUI
    root.mainloop()  # Run the application